In [1]:
import pandas as pd
import math
import numpy as np
import os

In [2]:
train_file='./input/tap_fun_train.csv'
test_file='./input//tap_fun_test.csv'
train_data=pd.read_csv(train_file)
test_data=pd.read_csv(test_file)

In [3]:
train_data_sample = train_data.sample(frac=0.1, random_state=42).copy()

In [4]:
def cal_na(dataframe):
    n_list = dataframe.isnull().sum(axis = 1).tolist()
    res = {col:n for col,n in zip(dataframe.columns, n_list)}
    return res

In [5]:
res_train = cal_na(train_data)
res_train

{'user_id': 0,
 'register_time': 0,
 'wood_add_value': 0,
 'wood_reduce_value': 0,
 'stone_add_value': 0,
 'stone_reduce_value': 0,
 'ivory_add_value': 0,
 'ivory_reduce_value': 0,
 'meat_add_value': 0,
 'meat_reduce_value': 0,
 'magic_add_value': 0,
 'magic_reduce_value': 0,
 'infantry_add_value': 0,
 'infantry_reduce_value': 0,
 'cavalry_add_value': 0,
 'cavalry_reduce_value': 0,
 'shaman_add_value': 0,
 'shaman_reduce_value': 0,
 'wound_infantry_add_value': 0,
 'wound_infantry_reduce_value': 0,
 'wound_cavalry_add_value': 0,
 'wound_cavalry_reduce_value': 0,
 'wound_shaman_add_value': 0,
 'wound_shaman_reduce_value': 0,
 'general_acceleration_add_value': 0,
 'general_acceleration_reduce_value': 0,
 'building_acceleration_add_value': 0,
 'building_acceleration_reduce_value': 0,
 'reaserch_acceleration_add_value': 0,
 'reaserch_acceleration_reduce_value': 0,
 'training_acceleration_add_value': 0,
 'training_acceleration_reduce_value': 0,
 'treatment_acceleraion_add_value': 0,
 'treatm

In [6]:
train_data.columns

Index(['user_id', 'register_time', 'wood_add_value', 'wood_reduce_value',
       'stone_add_value', 'stone_reduce_value', 'ivory_add_value',
       'ivory_reduce_value', 'meat_add_value', 'meat_reduce_value',
       ...
       'pvp_battle_count', 'pvp_lanch_count', 'pvp_win_count',
       'pve_battle_count', 'pve_lanch_count', 'pve_win_count',
       'avg_online_minutes', 'pay_price', 'pay_count', 'prediction_pay_price'],
      dtype='object', length=109)

In [7]:
cols = train_data.columns[2:-1].tolist()
X = train_data[cols].copy()
y = train_data['prediction_pay_price'].copy()

In [8]:
#特征工程
ts = pd.to_datetime(train_data['register_time'])

In [9]:
X['weekday'] = ts.dt.weekday
X['month'] = ts.dt.month
X['day'] = ts.dt.day
X['time'] = ts.dt.hour * 3600 + ts.dt.minute * 60 + ts.dt.second

In [10]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, y_train = X, y

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
# X_train_ = scaler.transform(X_train)
# X_test_ = scaler.transform(X_test)
X_train_ = scaler.transform(X_train)

In [26]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
parameters = {'n_estimators':[10,100,1000]}
reg = RandomForestRegressor(random_state=42,n_jobs=70)
clf = GridSearchCV(estimator=reg, param_grid=parameters, scoring='neg_mean_squared_error',
                   cv=3, n_jobs=70, refit=True, verbose=3)

In [ ]:
clf.fit(X_train_, y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=10 .................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=100 ................................................
[CV] n_estimators=1000 ...............................................
[CV] n_estimators=1000 ...............................................
[CV] n_estimators=1000 ...............................................


In [28]:
clf.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=70,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [33]:
clf.cv_results_

/data/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/data/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/data/miniconda2/envs/python3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/data/mini

{'mean_fit_time': array([ 3832.33247932,  6599.86188436, 23811.11174901]),
 'std_fit_time': array([786.84280161, 119.97995326,  97.88823533]),
 'mean_score_time': array([18.63440235, 16.79335006, 14.38614329]),
 'std_score_time': array([2.58265504, 3.41581353, 6.4108255 ]),
 'param_n_estimators': masked_array(data=[10, 100, 1000],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 10},
  {'n_estimators': 100},
  {'n_estimators': 1000}],
 'split0_test_score': array([-4919.38848706, -4551.10354216, -4412.39742854]),
 'split1_test_score': array([-4795.07261303, -4312.38918356, -4237.93166519]),
 'split2_test_score': array([-5078.12155401, -4562.78549865, -4593.33506013]),
 'mean_test_score': array([-4930.8608847 , -4475.42607479, -4414.55471795]),
 'std_test_score': array([115.83864547, 115.3830951 , 145.10084702]),
 'rank_test_score': array([3, 2, 1], dtype=int32),
 'split0_train_score': array([-601.8290747 , -550.11

In [34]:
X_pred = test_data[cols]
ts_ = pd.to_datetime(test_data['register_time'])
X_pred['weekday'] = ts_.dt.weekday
X_pred['month'] = ts_.dt.month
X_pred['day'] = ts_.dt.day
X_pred['time'] = ts_.dt.hour * 3600 + ts_.dt.minute * 60 + ts_.dt.second
X_pred_ = scaler.transform(X_pred)

/data/miniconda2/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [35]:
result = test_data[['user_id']].copy()
result['prediction_pay_price'] = clf.best_estimator_.predict(X_pred_)

In [36]:
result.to_csv('./output/submission.csv', index=False)